# Scraping Secretaría Senado

In [1]:
# Acceder a la web
import requests

# Manejo de texto HTML
from bs4 import BeautifulSoup
import re

#### Manejo de Datos ####
# Manejo de Tablas
import pandas as pd

In [2]:
def limpiar_texto(texto):
    # Quitar comillas sencillas y reemplazar por dobles
    texto_limpio = re.sub(r"[\\']",'\'',texto)
    # Quitar Espacios extra (dos o más)
    texto_limpio = re.sub('\s\s+', ' ',texto_limpio)
    # Quitar el patrón . , [texto]
    texto_limpio = re.sub('\.\s,\s', '. ',texto_limpio)
    texto_limpio = re.sub('\s\.', '.',texto_limpio)
    texto_limpio = re.sub('\s,', ',',texto_limpio)
    # Algunos países tienen prob
    texto_limpio = re.sub(u'\\\'xa0','',texto_limpio)
    # Quitar paréntesis angulares
    texto_limpio = re.sub('<|>','',texto_limpio)
    # Quitar espacios iniciales y finales
    texto_limpio = texto_limpio.strip()
    return texto_limpio

### Script general

In [3]:
def secretariasenado(sitio):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    base = 'http://www.secretariasenado.gov.co/senado/basedoc/'
    url = base+sitio+".html"
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    #Modificación del código HTML
    def modif(soup):
        artics = soup.select('p:not(.centrado)>a.bookmarkaj , p:not(.centrado)>span.b_aj>a')
        for artic in artics:
            if 'ARTICULO' in artic.text or 'ARTÍCULO' in artic.text:
                #Creando una agrupación para el artículo
                artic.parent.wrap(soup.new_tag("legend"))
                # Revisión de los siguientes hermanos
                obj = artic.parent.find_all_next(['p','div'])
                for ob in obj:
                    a = ob.select('a.bookmarkaj')
                    if len(ob.select('a.antsig')) != 0:#Fin de la página
                        break
                    if len(a) == 0:
                        crnt_lgnd = soup.select('legend')[-1]
                        crnt_lgnd.append(ob)
                    elif len(a) > 0:#Empieza un nuevo artículo
                        break

    numeros = []
    titulos = []
    textos = []
    estados = []
    informacion = []
    i = 1
    while True: # Ejecución dentro de cada una de las páginas de la ley
        modif(soup)
        arts = soup.select('legend')

        # Código javascript
        url_js = base+'js/'+re.sub('.html','.js',re.sub(base,'',url))
        res_js = requests.get(url_js,headers=headers)
        js = BeautifulSoup(res_js.content, 'html.parser')

        for art in arts:
            span = art.select('a.bookmarkaj,span.b_aj>a')[0].text

            splt = re.split(r'\.',span)

            # Título del artículo
            titulo = splt[1]
            titulo = limpiar_texto(titulo)
            titulos.append(titulo)

            # Número del artículo
            numero = splt[0]
            numero = limpiar_texto(numero)
            numeros.append(numero)

            # Texto del artículo
            texto = art.text
            texto = texto.replace(span,'')
            try:
                  comm = re.findall("<(.*?)>",texto)
                  for com in comm:
                    texto = texto.replace(com,'')
            except:
                comm = ''
            texto = re.sub("Ver |Jurisprudencia Vigencia|Legislación Anterior|Legislación anterior|Notas de Vigencia|Notas del Editor",'',texto)

            # Estado del artículo
            comm = ' '.join(comm)
            if comm != '' and ('erogado' in comm or 'lo INEXEQUIBLE' in comm):
                estado = 'Derogado'
                #texto = comm
            else:
                estado = 'Exequible'

            if 'tachado' in comm:
                for s in art.select('s'):
                    s.extract()
                texto, texto_anterior = art.text, texto
                texto = texto.replace(span,'')
                try:
                      comm = re.findall("<(.*?)>",texto)
                      for com in comm:
                        texto = texto.replace(com,'')
                except:
                    comm = ''
                texto = re.sub("Ver |Jurisprudencia Vigencia|Legislación Anterior|Legislación anterior|Notas de Vigencia|Notas del Editor",'',texto)
            texto = limpiar_texto(texto)
            textos.append(texto)
            estados.append(estado)

            # Comentarios
            infos = art.select('div>a.caja_vja_encabezado')
            informt = ''
            if len(infos)>0:
                for info in infos:
                    funct = re.sub('javascript:','',info.get('href'))
                    funct = re.sub('\(\)','',funct)
                    funct = funct+"\(\)\s\n{\s\n.*\n.*"
                    aux = re.findall(funct,str(js))[0]
                    intxt = BeautifulSoup(re.findall("<tbody>.*</tbody>",aux)[0], 'html.parser').text
                    informt = str(informt) + info.text + ' : ' + limpiar_texto(str(intxt)) + '\n'
            try:
                informt = str(informt) + 'Texto anterior: ' + limpiar_texto(str(texto_anterior)) + '\n'
                del(texto_anterior)
            except:
                pass
            informacion.append(informt)
        # Preparar la selección en la siguiente página
        antsig = soup.select('a.antsig')
        if len(antsig) == 2 and antsig[1].text == 'Anterior':
            break
        url = base+antsig[1].get('href')
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')

        # Respuesta cada 5 iteraciones
        if i % 5 == 0:
            print(f'Iteración {i}...',end=' ')
        i += 1
    # Creación del dataframe
    df = pd.DataFrame(zip(numeros,titulos,textos,estados,informacion),columns=['Número','Título','Texto','Estado','Informacion'])
    return df

In [4]:
df_codigo_civil = secretariasenado('codigo_civil')
df_codigo_civil.to_excel('codigo_civil.xlsx',index=False)
df_codigo_civil

Iteración 5... Iteración 10... Iteración 15... Iteración 20... Iteración 25... Iteración 30... Iteración 35... Iteración 40... Iteración 45... Iteración 50... Iteración 55... Iteración 60... Iteración 65... Iteración 70... Iteración 75... Iteración 80... 

,Número,Título,Texto,Estado,Informacion
0,ARTICULO 1o,DISPOSICIONES COMPRENDIDAS,El Código Civil comprende las disposiciones le...,Exequible,
1,ARTICULO 2o,APLICABILIDAD,En el presente Código Civil de la unión se reú...,Exequible,
2,ARTICULO 3o,OBLIGATORIEDAD,Considerado este Código en su conjunto en cada...,Exequible,
3,ARTICULO 4o,DEFINICION DE LEY,Ley es una declaración de la voluntad soberana...,Exequible,
4,ARTICULO 5o,PENAS EN LA LEY CIVIL,"Pero no es necesario que la ley que manda, pro...",Exequible,
...,...,...,...,...,...
2670,ARTICULO 2680,,Notas de vigencia,Derogado,Notas de vigencia : - Artículo derogado por el...
2671,ARTICULO 2681,,Notas de vigencia,Derogado,Notas de vigencia : - Artículo derogado por el...
2672,ARTICULO 2682,,Notas de vigencia,Derogado,Notas de vigencia : - Artículo derogado por el...
2673,ARTICULO 2683,,El presente Código comenzará a regir desde su ...,Exequible,


In [5]:
df_constitucion = secretariasenado('constitucion_politica_1991')
df_constitucion.to_excel('constitucion.xlsx',index=False)
df_constitucion

Iteración 5... Iteración 10... Iteración 15... 

,Número,Título,Texto,Estado,Informacion
0,ARTICULO 1o,,"Colombia es un Estado social de derecho, organ...",Exequible,Concordancias : Ley 388 de 1997; Art. 1o.; Art...
1,ARTICULO 2o,,Son fines esenciales del Estado: servir a la c...,Exequible,Concordancias : Ley 5 de 1992; Art. 230; Art. ...
2,ARTICULO 3o,,La soberanía reside exclusivamente en el puebl...,Exequible,Jurisprudencia Concordante : Corte Constitucio...
3,ARTICULO 4o,,La Constitución es norma de normas. En todo ca...,Exequible,Concordancias : Ley 393 de 1997; Art. 20\nConc...
4,ARTICULO 5o,,"El Estado reconoce, sin discriminación alguna,...",Exequible,Concordancias : Constitución Política; Art. 13...
...,...,...,...,...,...
464,ARTÍCULO TRANSITORIO 6o,FORMA DE ELECCIÓN,En cada una de las Circunscripciones Transitor...,Exequible,Notas de Vigencia : - Artículo adicionado por ...
465,ARTÍCULO TRANSITORIO 7o,FECHA DE ELECCIONES,Las elecciones de los Representantes a la Cáma...,Exequible,Notas de Vigencia : - Artículo adicionado por ...
466,ARTÍCULO TRANSITORIO 8o,FINANCIACIÓN,La financiación de las campañas será preponder...,Exequible,Notas de Vigencia : - Artículo adicionado por ...
467,ARTÍCULO TRANSITORIO 9o,ACCESO A MEDIOS DE COMUNICACIÓN,Cuando se utilicen medios de comunicación que ...,Exequible,Notas de Vigencia : - Artículo adicionado por ...


In [6]:
df_codigo_administrativo = secretariasenado('ley_1437_2011')
df_codigo_administrativo.to_excel('codigo_administrativo.xlsx',index=False)
df_codigo_administrativo

Iteración 5... 

,Número,Título,Texto,Estado,Informacion
0,ARTÍCULO 1o,FINALIDAD DE LA PARTE PRIMERA,Las normas de esta Parte Primera tienen como f...,Exequible,
1,ARTÍCULO 2o,ÁMBITO DE APLICACIÓN,Las disposiciones de esta Parte Primera no se ...,Exequible,
2,ARTÍCULO 3o,PRINCIPIOS,Todas las autoridades deberán interpretar y ap...,Exequible,
3,ARTÍCULO 4o,FORMAS DE INICIAR LAS ACTUACIONES ADMINISTRATIVAS,Las actuaciones administrativas podrán iniciar...,Exequible,
4,ARTÍCULO 5o,DERECHOS DE LAS PERSONAS ANTE LAS AUTORIDADES,En sus relaciones con las autoridades toda per...,Exequible,Notas de Vigencia : - Numeral modificado por e...
...,...,...,...,...,...
315,ARTÍCULO 305,IMPLANTACIÓN DEL NUEVO SISTEMA PROCESAL,Con el fin de conseguir la transición hacia la...,Exequible,
316,ARTÍCULO 306,ASPECTOS NO REGULADOS,En los aspectos no contemplados en este Código...,Exequible,
317,ARTÍCULO 307,"RECURSOS PARA LA IMPIMENTACIÓN SIC, IMPLEMENTA...",La implementación y desarrollo de la presente ...,Exequible,
318,ARTÍCULO 308,RÉGIMEN DE TRANSICIÓN Y VIGENCIA,El presente Código comenzará a regir el dos (2...,Exequible,
